In [5]:
console.log('*'.repeat(80))
console.log('Docker course'.padStart(45, ' '))
console.log('*'.repeat(80))

********************************************************************************
                                Docker course
********************************************************************************


docker run -d -p 80:80 docker/getting-started

In [ ]:
- docker container start 84c41e1776fc
- docker container stop 84c41e1776fc
- docker ps (-a)
- docker image ls
- docker container ls
- docker image ls
- Dockerfile file in root folder of app
- 
** dockerfile **
FROM node:12-alpine
WORKDIR /app
COPY . .
RUN yarn install --production
CMD ["node", "src/index.js"]

- docker build -t getting-started .
- docker run -dp 3000:3000 getting-started
- docker rm (-f) <the-container-id>
- docker tag getting-started YOUR-USER-NAME/getting-started
- docker push YOUR-USER-NAME/getting-started
-  Play with Docker.
- + ADD NEW INSTANCE
- docker run -dp 3000:3000 YOUR-USER-NAME/getting-started



- docker run -d ubuntu bash -c "shuf -i 1-10000 -n 1 -o /data.txt && tail -f /dev/null"
- docker exec < container-id > cat /data.txt
- docker run -it ubuntu ls /
- docker rm -f 

- docker volume create todo-db
- docker run -dp 3000:3000 -v todo-db:/etc/todos getting-started

- docker images -f dangling=true
- docker rmi 4e2eef94cd6b

- docker volume inspect todo-db

- docker run -dp 3000:3000 
    -w /app -v "$(pwd):/app" 
    node:12-alpine 
    sh -c "yarn install && yarn run dev"
- 

- docker logs -f < container-id >

- docker network create todo-app


- docker run --name some-mysql -e MYSQL_ROOT_PASSWORD=my-secret-pw -d mysql:tag
- docker run -it --network some-network --rm mysql mysql -hsome-mysql -uexample-user -p
- docker run -it --rm mysql mysql -hsome.mysql.host -usome-mysql-user -p

>> # Use root/example as user/password credentials
>>version: '3.1'

>>services:

  >>db:
    image: mysql
    command: --default-authentication-plugin=mysql_native_password
    restart: always
    environment:
      MYSQL_ROOT_PASSWORD: example

  >>adminer:
    image: adminer
    restart: always
    ports:
      - 8080:8080

docker-compose -f stack.yml up

In [ ]:
- docker exec -it some-mysql bash
- docker logs some-mysql

- https://hub.docker.com/_/mysql/

- docker exec -it < mysql-container-id > mysql -p
- mysql> SHOW DATABASES;

- docker run -it --network todo-app nicolaka/netshoot
- dig mysql




    MYSQL_HOST - the hostname for the running MySQL server
    MYSQL_USER - the username to use for the connection
    MYSQL_PASSWORD - the password to use for the connection
    MYSQL_DB - the database to use once connected


>>  docker run -dp 3000:3000 `
      -w /app -v "$(pwd):/app" `
      --network todo-app `
      -e MYSQL_HOST=mysql `
      -e MYSQL_USER=root `
      -e MYSQL_PASSWORD=secret `
      -e MYSQL_DB=todos `
      node:12-alpine `
      sh -c "yarn install && yarn run dev"


docker exec -ti < mysql-container-id > mysql -p todos

mysql> select * from todo_items;

172.18.0.2

docker-compose version

docker-compose.yml

version: "3.8"
services:

  redis:
    image: redis:alpine
    ports:
      - "6379"
    networks:
      - frontend
    deploy:
      replicas: 2
      update_config:
        parallelism: 2
        delay: 10s
      restart_policy:
        condition: on-failure

  db:
    image: postgres:9.4
    volumes:
      - db-data:/var/lib/postgresql/data
    networks:
      - backend
    deploy:
      placement:
        max_replicas_per_node: 1
        constraints:
          - "node.role==manager"

  vote:
    image: dockersamples/examplevotingapp_vote:before
    ports:
      - "5000:80"
    networks:
      - frontend
    depends_on:
      - redis
    deploy:
      replicas: 2
      update_config:
        parallelism: 2
      restart_policy:
        condition: on-failure

  result:
    image: dockersamples/examplevotingapp_result:before
    ports:
      - "5001:80"
    networks:
      - backend
    depends_on:
      - db
    deploy:
      replicas: 1
      update_config:
        parallelism: 2
        delay: 10s
      restart_policy:
        condition: on-failure

  worker:
    image: dockersamples/examplevotingapp_worker
    networks:
      - frontend
      - backend
    deploy:
      mode: replicated
      replicas: 1
      labels: [APP=VOTING]
      restart_policy:
        condition: on-failure
        delay: 10s
        max_attempts: 3
        window: 120s
      placement:
        constraints:
          - "node.role==manager"

  visualizer:
    image: dockersamples/visualizer:stable
    ports:
      - "8080:8080"
    stop_grace_period: 1m30s
    volumes:
      - "/var/run/docker.sock:/var/run/docker.sock"
    deploy:
      placement:
        constraints:
          - "node.role==manager"

networks:
  frontend:
  backend:

volumes:
  db-data:

version: "3.7"

services:


version: "3.7"

services:
  app:
    image: node:12-alpine
    command: sh -c "yarn install && yarn run dev"
    ports:
      - 3000:3000
    working_dir: /app
    volumes:
      - ./:/app
    environment:
      MYSQL_HOST: mysql
      MYSQL_USER: root
      MYSQL_PASSWORD: secret
      MYSQL_DB: todos

  mysql:
    image: mysql:5.7
    volumes:
      - todo-mysql-data:/var/lib/mysql
    environment: 
      MYSQL_ROOT_PASSWORD: secret
      MYSQL_DATABASE: todos

volumes:
  todo-mysql-data:

docker-compose logs -f

docker-compose down (--volumes)

docker image history --no-trunc getting-started

FROM node:12-alpine
WORKDIR /app
COPY package.json yarn.lock ./
RUN yarn install --production
COPY . .
CMD ["node", "src/index.js"]

.dockerignore >> node_modules

- Using cache
- Multi-Stage Builds


FROM maven AS build
WORKDIR /app
COPY . .
RUN mvn package

FROM tomcat
COPY --from=build /app/target/file.war /usr/local/tomcat/webapps 

FROM node:12 AS build
WORKDIR /app
COPY package* yarn.lock ./
RUN yarn install
COPY public ./public
COPY src ./src
RUN yarn run build

FROM nginx:alpine
COPY --from=build /app/build /usr/share/nginx/html

- Container Orchestration
- Cloud Native Computing Foundation Projects